Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

/home/anatoly/anaconda3/envs/mipt-trends2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


First reload the data we generated in `1_notmnist.ipynb`.

In [14]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save    # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [15]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Introduce helper function for calculating accuracy.

In [16]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
                    / predictions.shape[0])

|Progress bar to visualize accuracy.

In [30]:
from ipywidgets import IntProgress
from IPython.display import display

def progress(max):
    p = IntProgress(max = max)
    p.description = 'Running'
    display(p)
    return p

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

# We will start with logistic model

In [ ]:
l2_loss = 0.01
batch_size = 128
graph = tf.Graph()

with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_l2_loss = tf.placeholder(tf.float32)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf_l2_loss * tf.nn.l2_loss(weights)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let`s run it!

In [ ]:
num_steps = 3001
period_stat = 20

def plot(history):
    # summarize history for accuracy
    plt.plot(history['steps'], history['minibatch'])
    plt.plot(history['steps'], history['validation'])
    plt.plot(history['steps'], history['test'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['minibatch', 'validation', 'test'], loc='upper left')
    plt.show()

def train_logistic(l2_loss):
    print("Trainitng model with L2 regularization: %f" % l2_loss)
    history = {
        'minibatch' : [],
        'validation' : [],
        'test' : [],
        'steps' : []
    }

    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):

            # Pick an offset within the training data, which has been randomized.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # Prepare a dictionary telling the session where to feed the minibatch.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_l2_loss: l2_loss}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

            if (step % period_stat == 0):
                history["steps"].append(step)
                history["validation"].append(accuracy(valid_prediction.eval(), valid_labels))
                history["minibatch"].append(accuracy(predictions, batch_labels))
                history["test"].append(accuracy(test_prediction.eval(), test_labels))

        plot(history)
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

try_loss = [0.0001, 0.001, 0.01, 0.03, 0.1, 0.3]
for l2_loss in try_loss:
    train_logistic(l2_loss)

## Let's create a neural network with one hidden layer, RELU nonlinearity and optional l2 regularization and dropout.

Create tensorflow graph.

In [31]:
hidden_nodes = 1024
batch_size = 128

nn_with_hidden_layer = tf.Graph()

with nn_with_hidden_layer.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    l2 = tf.placeholder(tf.float32)
    p_dropout = tf.placeholder(tf.float32)
    
    # Variables
    
    # 1st layer
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    # hidden layer
    w2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    def model(X):
        # Add layer name scopes for better graph visualization
        with tf.name_scope("input_layer"):
            h = tf.nn.relu(tf.matmul(X, w1) + b1)
        with tf.name_scope("hidden_layer"):
            h = tf.nn.dropout(h, p_dropout)
            h2 = tf.matmul(h, w2) + b2
            return(h2)
    
    logits = model(tf_train_dataset)
    
    # Training computation
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    ) + l2 * tf.nn.l2_loss(w1) + l2 * tf.nn.l2_loss(w2)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Create training function.

In [34]:
num_steps = 3001
period_stat = 500

def train_neural_n(stats = False):
    print("Staring training with parameters")
    p = progress(num_steps)
    
    with tf.Session(graph=nn_with_hidden_layer) as session:
        tf.global_variables_initializer().run()
        for step in range(num_steps):

            # Pick an offset within the training data, which has been randomized.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

            # Generae minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # Dictionaries to pass to session.
            train_d = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, l2: 0.1, p_dropout: 0.8}
            stat_d = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, l2: 0.1, p_dropout: 1.0}
            
            _ = session.run(optimizer, feed_dict=train_d)
            
            p.value = step
            
            if (step % period_stat == 0) & (stats == True):
                l, train_pred, valid_pred = session.run([loss, train_prediction, valid_prediction], feed_dict=stat_d)
                print(
                    "Step {0}: \n loss: {1} \n minibatch acc: {2}% \n validation acc: {3}% \n".format(
                        step,
                        l,
                        accuracy(train_pred, batch_labels),
                        accuracy(valid_pred, valid_labels)
                    )
                )
        
        test_pred = session.run(test_prediction, feed_dict=stat_d)                                               
        print("Test accuracy: %.1f%%" % accuracy(test_pred, test_labels))

try_loss = [0.0001, 0.001, 0.01, 0.03, 0.1, 0.3]
for l2_loss in try_loss:
    train_neural_n(stats = False)

Staring training with parameters


KeyboardInterrupt: 

In [23]:

try_loss = [0.0001, 0.001, 0.01, 0.03, 0.1, 0.3]
for l2_loss in try_loss:
    train_neural_n(l2_loss)

Initialized


NameError: global name 'TRUE' is not defined

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

        global_step = tf.Variable(0)    # count the number of steps taken.
        learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
